In [2]:
import os
import src.sfm_agi.snippets.find_tie_points_for_sfm as ftpfs

project_fld = "/data/ATM/data_1/sfm/agi_projects/"
project_name = "another_matching_try_agi"

# create the path to the project
project_path = os.path.join(project_fld, project_name, project_name + ".psx")

img_folder = os.path.join(project_fld, project_name, 'data', 'images')
mask_folder = os.path.join(project_fld, project_name, 'data', 'masks_adapted')

tp_dict, conf_dict = ftpfs.find_tie_points_for_sfm(img_folder, mask_folder, "sequential")
print(tp_dict)
print(conf_dict)

  Iterate combinations


100%|██████████| 10/10 [00:57<00:00,  5.71s/it, 2 tie points between CA214832V0077 and CA214832V0075]   

{('CA214832V0076', 'CA214832V0077'): array([[4594.        , 8942.        ,  652.49996331, 8713.        ],
       [9122.        , 7883.        , 4750.        , 7619.50000326],
       [4699.        , 7145.        ,  750.49998249, 6923.        ],
       ...,
       [8825.        , 8327.        , 4403.        , 8063.        ],
       [7946.        , 8606.        , 3773.        , 8350.        ],
       [8581.        , 8452.        , 4201.        , 8187.        ]]), ('CA214832V0076', 'CA214832V0075'): array([[ 955.        , 8616.        , 5001.        , 8912.        ],
       [1328.        , 8688.        , 5282.49992315, 8977.49992315],
       [3379.        , 8372.        , 7562.        , 8656.49993434],
       ...,
       [1156.        , 4147.        , 5357.        , 4392.        ],
       [4760.        , 6092.        , 8722.50196498, 6344.49803502],
       [3453.        , 6586.        , 7725.50582942, 6863.49417058]]), ('CA214832V0078', 'CA214832V0077'): array([[3854.        , 3207.       

In [16]:
import numpy as np

def create_tracks(tp_dict, conf_dict):
    """
    Create tracks from tie-points and confidence values.

    Parameters:
    - tp_dict: A dictionary where keys are tuples (img1id, img2id), and values are numpy arrays of shape (N, 4),
               containing x1, y1, x2, y2 coordinates.
    - conf_dict: A dictionary with the same keys as tp_dict, where values are numpy arrays of confidence values.

    Returns:
    - tracks_array: A numpy array of shape (M, 5), where each row is [x1, y1, x2, y2, track_id].
    """
    # Initialize data structures
    keypoint_to_track = {}  # Map from key-point (image ID, quantized (x, y)) to track ID
    track_id_counter = 0    # Counter for assigning unique track IDs
    tracks = {}             # Map from track ID to set of key-points

    # Function to quantize coordinates to handle floating-point precision
    def quantize_coord(x, y, precision=1e-3):
        return (int(round(x / precision)), int(round(y / precision)))

    # Process each pair of images
    for key in tp_dict:
        # Unpack image IDs from the key
        img1id, img2id = key  # Both img1id and img2id are strings

        # Extract tie-points
        x1y1x2y2 = tp_dict[key]  # Shape: (N, 4)
        x1 = x1y1x2y2[:, 0]
        y1 = x1y1x2y2[:, 1]
        x2 = x1y1x2y2[:, 2]
        y2 = x1y1x2y2[:, 3]
        N = x1.shape[0]

        # Process each matched pair of key-points
        for k in range(N):
            x1k, y1k = x1[k], y1[k]
            x2k, y2k = x2[k], y2[k]

            # Quantize coordinates
            qx1k, qy1k = quantize_coord(x1k, y1k)
            qx2k, qy2k = quantize_coord(x2k, y2k)

            kp1 = (img1id, qx1k, qy1k)
            kp2 = (img2id, qx2k, qy2k)

            # Check if key-points are already associated with a track
            track1 = keypoint_to_track.get(kp1)
            track2 = keypoint_to_track.get(kp2)

            if track1 is not None and track2 is not None:
                if track1 != track2:
                    # Merge tracks
                    tracks[track1]['keypoints'].update(tracks[track2]['keypoints'])
                    for kp in tracks[track2]['keypoints']:
                        keypoint_to_track[kp] = track1
                    # Merge matches
                    tracks[track1]['matches'].extend(tracks[track2]['matches'])
                    del tracks[track2]
                current_track = track1
            elif track1 is not None:
                # Assign kp2 to track1
                keypoint_to_track[kp2] = track1
                tracks[track1]['keypoints'].add(kp2)
                tracks[track1]['matches'].append((img1id, x1k, y1k, img2id, x2k, y2k))
                current_track = track1
            elif track2 is not None:
                # Assign kp1 to track2
                keypoint_to_track[kp1] = track2
                tracks[track2]['keypoints'].add(kp1)
                tracks[track2]['matches'].append((img1id, x1k, y1k, img2id, x2k, y2k))
                current_track = track2
            else:
                # Create new track
                current_track = track_id_counter
                track_id_counter += 1
                keypoint_to_track[kp1] = current_track
                keypoint_to_track[kp2] = current_track
                tracks[current_track] = {
                    'keypoints': set([kp1, kp2]),
                    'matches': [(img1id, x1k, y1k, img2id, x2k, y2k)]
                }
    
    # Prepare the output array
    tracks_list = []
    for track_id, data in tracks.items():
        for match in data['matches']:
            img1id, x1k, y1k, img2id, x2k, y2k = match
            tracks_list.append([x1k, y1k, x2k, y2k, track_id])

    # Convert to numpy array
    tracks_array = np.array(tracks_list)

    print("HI")

    return tracks_array

In [17]:
tracks = create_tracks(tp_dict, conf_dict)
print(tracks)

HI
[[4.59400000e+03 8.94200000e+03 6.52499963e+02 8.71300000e+03
  0.00000000e+00]
 [4.59400000e+03 8.94200000e+03 8.47050806e+03 9.20599463e+03
  0.00000000e+00]
 [9.12200000e+03 7.88300000e+03 4.75000000e+03 7.61950000e+03
  1.00000000e+00]
 ...
 [9.07100000e+03 5.72700000e+03 4.80700000e+03 5.50900000e+03
  1.57340000e+04]
 [8.90300000e+03 9.13100000e+03 4.66500000e+03 8.90200000e+03
  1.57350000e+04]
 [8.28800000e+03 9.03000000e+03 4.33300000e+03 8.81800000e+03
  1.57360000e+04]]


In [18]:
# save np as csv
tracks_fld = os.path.join(project_fld, project_name)
tracks_path = os.path.join(tracks_fld, 'tracks.csv')
np.savetxt(tracks_path, tracks, delimiter=",")

In [2]:
# load the tracks csv as pandas
import pandas as pd
import os
import src.sfm_agi.snippets.convert_ply_files as cpf
project_fld = "/data/ATM/data_1/sfm/agi_projects/"
project_name = "tracking_test"


# create the path to the project
project_path = os.path.join(project_fld, project_name, project_name + ".files")
tracks_fld = os.path.join(project_path, '0', '0', 'point_cloud', 'point_cloud')

cpf.convert_ply_files(tracks_fld)
exit()

tracks_path = os.path.join(tracks_fld, 'tracks.txt')
tracks_path_new = os.path.join(tracks_fld, 'tracks.ply')
tracks_pd = pd.read_csv(tracks_path, header=None)

# replace all values in the first column with 1
tracks_pd.iloc[:, 0] = 1
tracks_pd.columns=['color']
print(tracks_pd)

import src.export.export_ply as ep
ep.export_ply(tracks_pd, tracks_path_new)


Converted /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/p2.ply to /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/p2.txt
Converted /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/p3.ply to /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/p3.txt
Converted /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/points0.ply to /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/points0.txt
Converted /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/tracks.ply to /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/tracks.txt
Converted /data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/

In [38]:
import src.sfm_agi.snippets.zip_folder as zp
project_path = os.path.join(project_fld, project_name, project_name + ".files")
path_pc_fld = os.path.join(project_path, "0", "0", "point_cloud", "point_cloud")
                          
output_zip_path = os.path.join(path_pc_fld, 'point_cloud.zip')
zp.zip_folder(path_pc_fld, output_zip_path, delete_files=False)


Zipping folder...
/data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud
/data/ATM/data_1/sfm/agi_projects/tracking_test/tracking_test.files/0/0/point_cloud/point_cloud/point_cloud.zip
<?xml version="1.0" encoding="UTF-8"?>
<point_cloud version="1.2.0">
  <params>
    <dataType>uint8</dataType>
    <bands>
      <band/>
    </bands>
  </params>
  <tracks path="tracks.ply" count="1354"/>
  <projections camera_id="0" path="p0.ply" count="430"/>
  <projections camera_id="1" path="p1.ply" count="677"/>
  <projections camera_id="2" path="p2.ply" count="933"/>
  <projections camera_id="3" path="p3.ply" count="678"/>
  <keypoints>
    <block path="points0.zip"/>
    <block path="points1.zip"/>
  </keypoints>
  <meta>
    <property name="Info/LastSavedDateTime" value="2024:10:01 15:05:25"/>
    <property name="Info/LastSavedSoftwareVersion" value="2.1.2.18358"/>
    <property name="Info/OriginalDateTime" value="2024:10:01 15:05:24"/>
    <property name=